# **NOTEBOOK 1: DATA ADCQUISITION, CLEANING AND PREPARATION**

In this notebook, data is going to be imported from a CSV file and prepared for later visualisation and analysis.

---

The data is in the context of a parcel delivery company that every day registers orders in its system for distribution on the next working day. Our goal is to predict how many packages will be registered tomorrow... but first we have to read the data and make all the preparations for this.

# Import libraries

In [24]:
import pandas as pd
import datetime as dt
import warnings
warnings.filterwarnings('ignore')

import holidays

This data was obtained from a private webservice of the company connected to its database. So they were obtained through this service and deposited in a SQL database in a virtual machine enabled for this purpose. Once deposited in the database, by means of a query executed in SQL Server Management Studio, they were obtained and deposited in a CSV that serves as a starting point for the present work.

So... let's read that file!

In [25]:
# url initial data
url = '../files/data/initial_data.csv'
# read with pandas
df_initial_parcels = pd.read_csv(url, sep=";", names=['date', 'n_parcels', 'weekday', 'monthday', 'month', 'year'], header=None, index_col='date')
df_initial_parcels.head()

,n_parcels,weekday,monthday,month,year
date,,,,,
2010-01-01,4.0,6,1,1,2010
2010-01-02,164.0,7,2,1,2010
2010-01-03,8.0,1,3,1,2010
2010-01-04,25345.0,2,4,1,2010
2010-01-05,20130.0,3,5,1,2010


In [26]:
df_initial_parcels.tail()

,n_parcels,weekday,monthday,month,year
date,,,,,
2022-03-27,56338.0,1,27,3,2022
2022-03-28,417314.0,2,28,3,2022
2022-03-29,358210.0,3,29,3,2022
2022-03-30,331948.0,4,30,3,2022
2022-03-31,336889.0,5,31,3,2022


In [27]:
df_initial_parcels.dtypes

n_parcels    float64
weekday        int64
monthday       int64
month          int64
year           int64
dtype: object

We can see that we have for each of the days the total shipments recorded between January 1, 2010 and March 31, 2022. In addition, there are columns with the day of the week, the day of the month, the month and the year. 

We know that there are all days in the series because the query was implemented to do so, so we will have to check for null values later.

Now let's add some more for plotting graphs late (date as columns too and yearday).

In [28]:
# index to datetime
df_initial_parcels.index = pd.to_datetime(df_initial_parcels.index)

# add date and yearday columns
df_initial_parcels['date_c'] = df_initial_parcels.index
df_initial_parcels['yearday'] = df_initial_parcels.index.dayofyear
df_initial_parcels.head()

,n_parcels,weekday,monthday,month,year,date_c,yearday
date,,,,,,,
2010-01-01,4.0,6,1,1,2010,2010-01-01,1
2010-01-02,164.0,7,2,1,2010,2010-01-02,2
2010-01-03,8.0,1,3,1,2010,2010-01-03,3
2010-01-04,25345.0,2,4,1,2010,2010-01-04,4
2010-01-05,20130.0,3,5,1,2010,2010-01-05,5


# Generate holidays

As we are dealing with a time series and by the very nature of the company, we can guess that we will need to identify holidays and important days that may have strange behaviors. These are national holidays, Black Friday, Black Friday week, Christmas and the period of confinement and months after.

First, we are going to get the national holidays using the "holidays" library and we are going to add Holy Thursday, Black Friday and the Black Friday week.

In [29]:
# import national holidays
es_holidays = holidays.ES(years=(2010, 2011, 2012, 2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022))

# create Holy Thursday
es_holidays[dt.date(2010, 4, 1)] = 'Jueves Santo'
es_holidays[dt.date(2011, 4, 21)] = 'Jueves Santo'
es_holidays[dt.date(2012, 4, 5)] = 'Jueves Santo'
es_holidays[dt.date(2013, 3, 28)] = 'Jueves Santo'
es_holidays[dt.date(2014, 4, 17)] = 'Jueves Santo'
es_holidays[dt.date(2015, 4, 2)] = 'Jueves Santo'
es_holidays[dt.date(2016, 3, 24)] = 'Jueves Santo'
es_holidays[dt.date(2017, 4, 13)] = 'Jueves Santo'
es_holidays[dt.date(2018, 4, 29)] = 'Jueves Santo'
es_holidays[dt.date(2019, 4, 18)] = 'Jueves Santo'
es_holidays[dt.date(2020, 4, 9)] = 'Jueves Santo'
es_holidays[dt.date(2021, 4, 1)] = 'Jueves Santo'
es_holidays[dt.date(2022, 4, 14)] = 'Jueves Santo'

# create Black Friday
es_blackfriday = {}
es_blackfriday[dt.date(2010, 11, 29)] = 'Black friday'
es_blackfriday[dt.date(2011, 11, 28)] = 'Black friday'
es_blackfriday[dt.date(2012, 11, 26)] = 'Black friday'
es_blackfriday[dt.date(2013, 12, 2)] = 'Black friday'
es_blackfriday[dt.date(2014, 12, 1)] = 'Black friday'
es_blackfriday[dt.date(2015, 11, 30)] = 'Black friday'
es_blackfriday[dt.date(2016, 11, 28)] = 'Black friday'
es_blackfriday[dt.date(2017, 11, 27)] = 'Black friday'
es_blackfriday[dt.date(2018, 11, 26)] = 'Black friday'
es_blackfriday[dt.date(2019, 12, 2)] = 'Black friday'
es_blackfriday[dt.date(2020, 11, 30)] = 'Black friday'
es_blackfriday[dt.date(2021, 11, 29)] = 'Black friday'
es_blackfriday[dt.date(2022, 11, 28)] = 'Black friday'

# create Black Friday week
es_blackfridayWeek = {}
es_blackfridayWeek[dt.date(2010, 11, 26)] = 'Black friday week'
es_blackfridayWeek[dt.date(2010, 11, 27)] = 'Black friday week'
es_blackfridayWeek[dt.date(2010, 11, 28)] = 'Black friday week'
es_blackfridayWeek[dt.date(2010, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2010, 11, 30)] = 'Black friday week'
es_blackfridayWeek[dt.date(2010, 12, 1)] = 'Black friday week'
es_blackfridayWeek[dt.date(2010, 12, 2)] = 'Black friday week'
es_blackfridayWeek[dt.date(2011, 11, 25)] = 'Black friday week'
es_blackfridayWeek[dt.date(2011, 11, 26)] = 'Black friday week'
es_blackfridayWeek[dt.date(2011, 11, 27)] = 'Black friday week'
es_blackfridayWeek[dt.date(2011, 11, 28)] = 'Black friday week'
es_blackfridayWeek[dt.date(2011, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2011, 11, 30)] = 'Black friday week'
es_blackfridayWeek[dt.date(2011, 12, 1)] = 'Black friday week'
es_blackfridayWeek[dt.date(2012, 11, 23)] = 'Black friday week'
es_blackfridayWeek[dt.date(2012, 11, 24)] = 'Black friday week'
es_blackfridayWeek[dt.date(2012, 11, 25)] = 'Black friday week'
es_blackfridayWeek[dt.date(2012, 11, 26)] = 'Black friday week'
es_blackfridayWeek[dt.date(2012, 11, 27)] = 'Black friday week'
es_blackfridayWeek[dt.date(2012, 11, 28)] = 'Black friday week'
es_blackfridayWeek[dt.date(2012, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2013, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2013, 11, 30)] = 'Black friday week'
es_blackfridayWeek[dt.date(2013, 12, 1)] = 'Black friday week'
es_blackfridayWeek[dt.date(2013, 12, 2)] = 'Black friday week'
es_blackfridayWeek[dt.date(2013, 12, 3)] = 'Black friday week'
es_blackfridayWeek[dt.date(2013, 12, 4)] = 'Black friday week'
es_blackfridayWeek[dt.date(2013, 12, 5)] = 'Black friday week'
es_blackfridayWeek[dt.date(2014, 11, 28)] = 'Black friday week'
es_blackfridayWeek[dt.date(2014, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2014, 11, 30)] = 'Black friday week'
es_blackfridayWeek[dt.date(2014, 12, 1)] = 'Black friday week'
es_blackfridayWeek[dt.date(2014, 12, 2)] = 'Black friday week'
es_blackfridayWeek[dt.date(2014, 12, 3)] = 'Black friday week'
es_blackfridayWeek[dt.date(2014, 12, 4)] = 'Black friday week'
es_blackfridayWeek[dt.date(2015, 11, 27)] = 'Black friday week'
es_blackfridayWeek[dt.date(2015, 11, 28)] = 'Black friday week'
es_blackfridayWeek[dt.date(2015, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2015, 11, 30)] = 'Black friday week'
es_blackfridayWeek[dt.date(2015, 12, 1)] = 'Black friday week'
es_blackfridayWeek[dt.date(2015, 12, 2)] = 'Black friday week'
es_blackfridayWeek[dt.date(2015, 12, 3)] = 'Black friday week'
es_blackfridayWeek[dt.date(2016, 11, 25)] = 'Black friday week'
es_blackfridayWeek[dt.date(2016, 11, 26)] = 'Black friday week'
es_blackfridayWeek[dt.date(2016, 11, 27)] = 'Black friday week'
es_blackfridayWeek[dt.date(2016, 11, 28)] = 'Black friday week'
es_blackfridayWeek[dt.date(2016, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2016, 11, 30)] = 'Black friday week'
es_blackfridayWeek[dt.date(2016, 12, 1)] = 'Black friday week'
es_blackfridayWeek[dt.date(2017, 11, 24)] = 'Black friday week'
es_blackfridayWeek[dt.date(2017, 11, 25)] = 'Black friday week'
es_blackfridayWeek[dt.date(2017, 11, 26)] = 'Black friday week'
es_blackfridayWeek[dt.date(2017, 11, 27)] = 'Black friday week'
es_blackfridayWeek[dt.date(2017, 11, 28)] = 'Black friday week'
es_blackfridayWeek[dt.date(2017, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2017, 11, 30)] = 'Black friday week'
es_blackfridayWeek[dt.date(2018, 11, 23)] = 'Black friday week'
es_blackfridayWeek[dt.date(2018, 11, 24)] = 'Black friday week'
es_blackfridayWeek[dt.date(2018, 11, 25)] = 'Black friday week'
es_blackfridayWeek[dt.date(2018, 11, 26)] = 'Black friday week'
es_blackfridayWeek[dt.date(2018, 11, 27)] = 'Black friday week'
es_blackfridayWeek[dt.date(2018, 11, 28)] = 'Black friday week'
es_blackfridayWeek[dt.date(2018, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2019, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2019, 11, 30)] = 'Black friday week'
es_blackfridayWeek[dt.date(2019, 12, 1)] = 'Black friday week'
es_blackfridayWeek[dt.date(2019, 12, 2)] = 'Black friday week'
es_blackfridayWeek[dt.date(2019, 12, 3)] = 'Black friday week'
es_blackfridayWeek[dt.date(2019, 12, 4)] = 'Black friday week'
es_blackfridayWeek[dt.date(2019, 12, 5)] = 'Black friday week'
es_blackfridayWeek[dt.date(2020, 11, 27)] = 'Black friday week'
es_blackfridayWeek[dt.date(2020, 11, 28)] = 'Black friday week'
es_blackfridayWeek[dt.date(2020, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2020, 11, 30)] = 'Black friday week'
es_blackfridayWeek[dt.date(2020, 12, 1)] = 'Black friday week'
es_blackfridayWeek[dt.date(2020, 12, 2)] = 'Black friday week'
es_blackfridayWeek[dt.date(2020, 12, 3)] = 'Black friday week'
es_blackfridayWeek[dt.date(2021, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2021, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2021, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2021, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2021, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2021, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2021, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2022, 11, 25)] = 'Black friday week'
es_blackfridayWeek[dt.date(2022, 11, 26)] = 'Black friday week'
es_blackfridayWeek[dt.date(2022, 11, 27)] = 'Black friday week'
es_blackfridayWeek[dt.date(2022, 11, 28)] = 'Black friday week'
es_blackfridayWeek[dt.date(2022, 11, 29)] = 'Black friday week'
es_blackfridayWeek[dt.date(2022, 11, 30)] = 'Black friday week'
es_blackfridayWeek[dt.date(2022, 12, 1)] = 'Black friday week'

We also create a function to generate Christmas days.

In [30]:
# create Christmas
es_Christmas = {}
def christmas(df):
  if (df['month'] == 12 and df['monthday'] >= 10) or (df['month'] == 1 and df['monthday'] <= 6):
    es_Christmas[dt.date(df['year'], df['month'], df['monthday'])] = 'Christmas'
    return True
  else: 
    return False

Now we create a function to add to our dataset new columns with the weekends, black friday, holidays, Christmas, lockout period... and then we add this aditional data to the dataframe

So we are going to add boolean columns for the calendar data with the following:

* National holidays: using the holidays module, the Spanish national holidays are imported and others, such as Maundy Thursday, are added manually. 
* Black Friday: the Black Friday day is manually created for each of the years (the day with the most package registrations each year).
* BlackFriday week: another category is created that includes the 3 days before and after due to the high volume of these dates.
* Christmas: period corresponding to the second half of December and up to January 6.
* COVID: period from March 8 to September 1 due to the confinement and subsequent deconfinement until the return to relative normality.

In [31]:
# type day function
def type_day(df):
    if df['blackFriday']:
        return 'blackFriday'
    elif df['blackFridayWeek']:
        return 'blackFridayWeek'
    elif df['Christmas']:
        return 'Christmas'
    elif df['fest']:
        return 'fest'
    elif df['saturday']:
        return 'saturday'
    elif df['sunday']:
        return 'sunday'
    elif df['COVID']:
        return 'COVID'
    elif df['eve']:
        return 'eve'
    elif df['postFest']:
        return 'postFest'
    elif df['workable']:
        return 'workable'
    else:
        return 'workable'

In [32]:
df_daytype_parcels = df_initial_parcels

# add calendar data to dataset
df_daytype_parcels['fest'] = df_daytype_parcels.apply(lambda x: True if x['date_c'] in es_holidays else False, axis=1)
df_daytype_parcels['eve'] = df_daytype_parcels.apply(lambda x: True if x['date_c'] + pd.Timedelta(days=1) in es_holidays else False, axis=1)
df_daytype_parcels['postFest'] = df_daytype_parcels.apply(lambda x: True if x['date_c'] + pd.Timedelta(days=-1) in es_holidays else False, axis=1)
df_daytype_parcels['saturday'] = df_daytype_parcels.apply(lambda x: True if x['date_c'].weekday() == 5 else False, axis=1)
df_daytype_parcels['sunday'] = df_daytype_parcels.apply(lambda x: True if x['date_c'].weekday() == 6 else False, axis=1)
df_daytype_parcels['weekend'] = df_daytype_parcels.apply(lambda x: True if x['date_c'].weekday() > 4 else False, axis=1)
df_daytype_parcels['workable'] = ~(df_daytype_parcels['fest'] | df_initial_parcels['weekend'])
df_daytype_parcels['blackFriday'] = df_daytype_parcels.apply(lambda x: True if x['date_c'] in list(es_blackfriday.keys()) else False, axis=1)
df_daytype_parcels['blackFridayWeek'] = df_daytype_parcels.apply(lambda x: True if x['date_c'] in list(es_blackfridayWeek.keys()) else False, axis=1)
df_daytype_parcels['COVID'] = df_daytype_parcels.apply(lambda x: True if x['date_c'] >= dt.datetime.strptime('2020-03-08', '%Y-%m-%d') and x['date_c'] < dt.datetime.strptime('2020-09-01', '%Y-%m-%d')  else False, axis=1)
df_daytype_parcels['Christmas'] = df_daytype_parcels.apply(lambda x: christmas(x), axis=1)
df_daytype_parcels['typeDay']  = df_daytype_parcels.apply(lambda x: type_day(x), axis=1)

df_daytype_parcels.head()

,n_parcels,weekday,monthday,month,year,date_c,yearday,fest,eve,postFest,saturday,sunday,weekend,workable,blackFriday,blackFridayWeek,COVID,Christmas,typeDay
date,,,,,,,,,,,,,,,,,,,
2010-01-01,4.0,6,1,1,2010,2010-01-01,1,True,False,False,False,False,False,False,False,False,False,True,Christmas
2010-01-02,164.0,7,2,1,2010,2010-01-02,2,False,False,True,True,False,True,False,False,False,False,True,Christmas
2010-01-03,8.0,1,3,1,2010,2010-01-03,3,False,False,False,False,True,True,False,False,False,False,True,Christmas
2010-01-04,25345.0,2,4,1,2010,2010-01-04,4,False,False,False,False,False,False,True,False,False,False,True,Christmas
2010-01-05,20130.0,3,5,1,2010,2010-01-05,5,False,True,False,False,False,False,True,False,False,False,True,Christmas


As mentioned before, we have to check if there are nulls in our data

In [33]:
# check NA
df_daytype_parcels['n_parcels'].isna().sum()

48

There are 48 NAs, this is normal because on weekends and holidays there can be days without parcels and we know that these days are this way. So we will replace them with 0. Actually, these days did not exist in the data provided by the company and contained in the database, but at the time of obtaining the data, the query was implemented to generate those days that did not exist and now we find them as null values.

In [34]:
# fillna in "n_parcels" with 0
df_daytype_parcels['n_parcels'] = df_daytype_parcels['n_parcels'].fillna(0)
df_daytype_parcels['n_parcels'].isna().sum()

0

Finally, we have as a result the dataframe with all the days between the first and the last one, the number of parcels of each day and some columns with calendar data that will help us visualize, analyze and obtain important variables in the next steps.

So, let's save this dataframe and go on in the next nootbook.

In [35]:
# save data
df_daytype_parcels.to_csv("../files/data/daytype_parcels.csv", header=True)